In [1]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

In [ ]:
# prompt
# так , мне надо создать БД используя библиотеку FAKER смотри база данных собой представляет компанию IT где есть разные отделы , связисты (телефония, сетевые розетки) 8 человек; техническая поддержка пользователей ПК (помощь в установки ПО, исправления багов, добавления принторов и тд) 14 человек ; отдел информационной безопасности 8 человек; отдел инфраструктурных систем 6 человек отдел сопровождения прикладных систем 10 человек кабельщики 8 человек; так , я прошу тебя продумать к каждому отделу свои заявки , в плане что бы их было не мало в каждом отделе напрмер , у связистов пусть будет штук 10 , а у остальных отделов от 15 до 30 на твое усмотрение , прошу напряч все ресурсы которые ты можешь вот тебе поля для таблиц , таблицы будет 3 id_department идентификатор департамента phone_number рабочий телефон (начинается на 89---------) email рабочая почта position должность is_active активен ли сотрудник departments id_deparment идентификатор департамента name_deparment название департамента events id_event уникальный идентификатор title краткое описание проблемы (VARCHAR) description подробное описание (TEXT) created_at дата и время создания (DATETIME) deadline "Срок, до которого нужно выполнить заявку (плановое завершение) " completed_at Фактическое время выполнения (все сделано) closed_at Время, когда заявка была официально закрыта (могут быть разные с completed_at) status статус (например: “новая”, “в работе”, “выполнена”, “отклонена”) priority приоритет (например: “низкий”, “средний”, “высокий”) assigned_to сотрудник, назначенный на заявку (FK → employees.id) reported_by кто создал заявку (может быть внешний человек или сотрудник) id_department отдел, которому назначена заявка (FK → departments.id)

In [12]:
import csv
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('ru_RU')

# --- Departments ---
departments = [
    (1, "Связисты"),
    (2, "Техподдержка"),
    (3, "Информационная безопасность"),
    (4, "Инфраструктурные системы"),
    (5, "Сопровождение прикладных систем"),
    (6, "Кабельщики")
]

# Кол-во сотрудников по отделам
department_sizes = {
    1: 8,
    2: 14,
    3: 8,
    4: 6,
    5: 10,
    6: 8
}

# --- Positions by department ---
positions = {
    1: ["Инженер связи", "Специалист по телефонии", "Техник связи", "Системный инженер АТС"],
    2: ["Инженер технической поддержки", "Системный администратор", "Специалист helpdesk", "Техник по обслуживанию ПК"],
    3: ["Аналитик ИБ", "Инженер по ИБ", "Специалист SOC", "Администратор средств защиты информации"],
    4: ["Системный администратор", "Инженер серверных систем", "Администратор баз данных", "Специалист по виртуализации"],
    5: ["Системный аналитик", "Специалист по сопровождению 1С", "Администратор приложений", "Инженер по интеграции"],
    6: ["Кабельщик-сварщик", "Монтажник СКС", "Техник по обслуживанию сетей", "Специалист по прокладке кабеля"]
}

# --- Ticket templates (расширенные для 2–5 департаментов) ---
ticket_templates = {
    1: [("Не работает телефон", "Заменен кабель, восстановлена связь"),
        ("Нет сигнала в розетке", "Перепатчена розетка на коммутаторе"),
        ("Шумы в телефоне", "Заменена гарнитура"),
        ("Не проходит вызов", "Проверен шлюз, ошибка устранена"),
        ("Не работает внутренний номер", "Переименован в АТС, восстановлен доступ")],
    2: [("Не подключается сетевой диск", "Подключение восстановлено"),
        ("Не обновляется антивирус", "База обновлена вручную"),
        ("Завис компьютер", "Сбой устранен, диск проверен"),
        ("Не работает Teams", "Переустановлено приложение"),
        ("Ошибка печати", "Установлен новый драйвер принтера"),
        ("Сбой Outlook", "Исправлен профиль почты"),
        ("Не открывается браузер", "Очищен кеш и переустановлено приложение"),
        ("Монитор не включается", "Заменен кабель питания"),
        ("Ошибка учетной записи", "Аккаунт разблокирован"),
        ("Не работает мышь", "Заменено устройство"),
        ("Долгая загрузка", "Отключены автозагрузки"),
        ("Zoom не запускается", "Обновлена версия клиента"),
        ("ПК перегревается", "Проведена чистка от пыли"),
        ("Не работает принтер по Wi-Fi", "Настроено сетевое подключение"),
        ("Ошибка синхронизации OneDrive", "Перезапущен клиент"),
        ("Не работает камера в Zoom", "Обновлены драйверы"),
        ("Синий экран Windows", "Выполнено обновление драйверов"),
        ("USB устройство не определяется", "Переустановлены драйверы контроллера"),
        ("Неверное время на компьютере", "Синхронизированы часы через NTP"),
        ("Не подключается Wi-Fi", "Сброшены настройки адаптера"),
        ("Тормозит браузер", "Очищен кеш, отключены расширения"),
        ("Проблемы с VPN", "Заново настроен клиент"),
        ("Автоматически перезагружается ПК", "Проверена система охлаждения"),
        ("Проблемы с клавиатурой", "Заменено устройство"),
        ("Некорректная работа Excel", "Переустановлен Office"),
        ("Не открываются PDF файлы", "Установлен новый ридер")],
    3: [("Вход с неизвестного устройства", "Доступ временно заблокирован"),
        ("Попытка подбора пароля", "IP заблокирован"),
        ("Нарушение политики паролей", "Сотрудник проинструктирован"),
        ("Обнаружен троян", "ОС переустановлена"),
        ("Не прошёл аудит логов", "Настроено централизованное логирование"),
        ("Подозрение на утечку", "Проведена проверка каналов"),
        ("VPN не авторизуется", "Заменены ключи доступа"),
        ("Нарушение прав доступа", "Исправлены ACL"),
        ("Подозрительная активность", "Учётная запись заблокирована"),
        ("Несанкционированное ПО", "Программа удалена"),
        ("Сбой обновления антивируса", "Ручное обновление базы"),
        ("Фишинговая рассылка", "Домен добавлен в blacklist"),
        ("Обнаружена подозрительная программа", "Удалено и отправлено на анализ"),
        ("Несанкционированное подключение к Wi-Fi", "Сеть заблокирована"),
        ("Утечка данных через почту", "Отправка ограничена, проведено расследование"),
        ("Массовая рассылка спама с внутреннего ящика", "Учетка заблокирована"),
        ("Обнаружен руткит", "Система переустановлена"),
        ("Нарушение разграничения доступа", "Исправлены роли пользователей"),
        ("Атака на веб-приложение", "Включена WAF защита"),
        ("Брутфорс учетных записей", "Введен captcha"),
        ("Срабатывание DLP", "Доступ ограничен"),
        ("Обнаружен кейлоггер", "Устройство изолировано"),
        ("Удаленное подключение без согласования", "Сессия завершена, инцидент зафиксирован"),
        ("Сертификат безопасности просрочен", "Выпущен новый"),
        ("Аномальная активность в журнале", "Выполнена проверка событий")],
    4: [("Сбой контроллера домена", "Сервер перезагружен, доступ восстановлен"),
        ("DNS не отвечает", "Служба перезапущена"),
        ("DHCP раздаёт неверные IP", "Исправлен пул адресов"),
        ("Не проходит логин", "Синхронизирован каталог AD"),
        ("Медленный файловый сервер", "Перераспределены ресурсы"),
        ("Сбой SAN", "Заменен неисправный диск"),
        ("Почтовый сервер не отвечает", "Перезапущен сервис"),
        ("Резервное копирование не проходит", "Расширен диск бэкап-сервера"),
        ("Сбой гипервизора", "ВМ перенесены"),
        ("Ошибка в кластере", "Восстановлена нода"),
        ("VPN нестабилен", "Обновлены сертификаты"),
        ("Мониторинг не работал", "Агент перезапущен"),
        ("Нехватка места на сервере", "Расширен диск"),
        ("Сбой RAID-массива", "Восстановлен из резерва"),
        ("Проблемы с репликацией AD", "Восстановлена синхронизация"),
        ("Сервер зависает", "Установлены обновления"),
        ("Ошибка в системе виртуализации", "Перезапущен сервис"),
        ("Медленный доступ к БД", "Оптимизированы запросы"),
        ("Ошибка лицензирования", "Добавлены новые лицензии"),
        ("Сбой в балансировщике нагрузки", "Перезапущен сервис"),
        ("Аппаратный сбой сервера", "Заменен блок питания"),
        ("Не работают групповые политики", "Исправлены объекты GPO"),
        ("Проблемы с репликацией DFS", "Проведена синхронизация"),
        ("Не запускается служба Exchange", "Служба перезапущена")],
    5: [("Ошибка выгрузки в Excel", "Исправлена обработка"),
        ("Ошибка расчета зарплаты", "Обновлен регламент"),
        ("Ошибка при счете", "Исправлена форма документа"),
        ("Некорректный курс валюты", "Данные обновлены"),
        ("CRM не отправляет письма", "SMTP проверен, доступ восстановлен"),
        ("Не формируется отчет", "Переписан запрос"),
        ("Ошибка авторизации", "Обновлены роли"),
        ("Зависание при выгрузке", "Оптимизирована интеграция"),
        ("Некорректные данные", "Исправлены справочники"),
        ("Ошибка ЭДО", "Установлен сертификат"),
        ("Не отображаются товары", "Исправлен обмен"),
        ("Задвоение заказов", "Исправлен бизнес-процесс"),
        ("Неверные права в CRM", "Изменены ACL"),
        ("Ошибка при закрытии месяца", "Обновлен модуль"),
        ("Некорректный расчет налогов", "Исправлен алгоритм"),
        ("Не загружаются курсы валют", "Настроен обмен"),
        ("Ошибка в отчетности", "Обновлен шаблон"),
        ("Не работает интеграция с банком", "Исправлен коннектор"),
        ("Задержка при формировании отчета", "Оптимизирован запрос"),
        ("Проблемы с выгрузкой в XML", "Исправлен формат"),
        ("Не отображаются графики", "Добавлена библиотека"),
        ("Некорректная авторизация в CRM", "Исправлена схема SSO"),
        ("Ошибка импорта данных", "Исправлен парсер"),
        ("Не отправляются уведомления", "Исправлена очередь сообщений"),
        ("Проблемы с ERP", "Перезапущен сервис"),
        ("Некорректный расчет себестоимости", "Исправлена формула"),
        ("Ошибка обмена с маркетплейсом", "Исправлен модуль интеграции"),
        ("Сбой при создании документа", "Исправлена форма")],
    6: [("Не работает интернет в кабинете", "Заменен сетевой кабель"),
        ("Оборван кабель в шахте", "Проведена новая линия"),
        ("Нет связи на этаже", "Подключен резервный кабель"),
        ("Кабель поврежден", "Произведен ремонт"),
        ("Плохой сигнал", "Заменен патч-корд")]
}

# --- Generate Employees ---
employees = []
emp_id = 1
for dep_id, dep_name in departments:
    for _ in range(department_sizes[dep_id]):
        employees.append({
            "id": emp_id,
            "id_department": dep_id,
            "phone_number": fake.msisdn()[:11],
            "email": fake.email(),
            "position": random.choice(positions[dep_id]),
            "is_active": random.choice([True, True, True, False])
        })
        emp_id += 1

# --- Generate pool of reporters (один раз для всей базы) ---
reporters = [fake.name() for _ in range(1500)]

# --- Generate Events ---
events = []

TOTAL_EVENTS = 500

for event_id in range(1, TOTAL_EVENTS + 1):
    dep_id, dep_name = random.choice(departments)
    title, description = random.choice(ticket_templates[dep_id])

    created_at = fake.date_time_between(start_date="-12M", end_date="now")
    deadline = created_at + timedelta(days=random.randint(1, 14))

    status = None
    completed_at = None
    closed_at = None

    if random.random() < 0.1:
        status = "в работе"
    else:
        completed_at = created_at + timedelta(hours=random.randint(2, 24*14))
        closed_at = completed_at + timedelta(hours=random.randint(1, 72))

        if random.random() < 0.1:
            status = "отклонена"
        else:
            if completed_at <= deadline:
                status = "выполнена"
            else:
                status = "просроченная"

    # вот тут добавляем приоритет
    priority = random.choice(["низкий", "средний", "высокий"])

    events.append({
        "id_event": event_id,
        "title": title,
        "description": description,
        "created_at": created_at,
        "deadline": deadline,
        "completed_at": completed_at,
        "closed_at": closed_at,
        "status": status,
        "priority": priority,
        "assigned_to": random.choice(employees)["id"],
        "reported_by": random.choice(reporters),  # заказчики теперь повторяются
        "id_department": dep_id
    })
   

# --- Save CSV ---
with open("departments.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_department", "name_department"])
    for dep_id, dep_name in departments:
        writer.writerow([dep_id, dep_name])

with open("employees.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "id_department", "phone_number", "email", "position", "is_active"])
    for e in employees:
        writer.writerow([e["id"], e["id_department"], e["phone_number"], e["email"], e["position"], e["is_active"]])

with open("events.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_event", "title", "description", "created_at", "deadline", "completed_at", "closed_at", "status", "priority", "assigned_to", "reported_by", "id_department"])
    for ev in events:
        writer.writerow([ev["id_event"], ev["title"], ev["description"], ev["created_at"], ev["deadline"], ev["completed_at"], ev["closed_at"], ev["status"], ev["priority"], ev["assigned_to"], ev["reported_by"], ev["id_department"]])


In [6]:
# empl = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/employees.csv') # MAC
empl = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\employees.csv') # WIN

In [3]:
depart = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/departments.csv') # MAC
depart = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\departments.csv') # WIN

In [4]:
events = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/events.csv') # MAC
events = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\events.csv') # WIN

In [5]:
empl_depart = pd.merge(empl, depart, on='id_department', how='inner')

In [6]:
empl_depart.groupby(['name_department', 'position'])['position'].count()

name_department                  position                               
Информационная безопасность      Администратор средств защиты информации    3
                                 Аналитик ИБ                                2
                                 Инженер по ИБ                              2
                                 Специалист SOC                             1
Инфраструктурные системы         Администратор баз данных                   2
                                 Инженер серверных систем                   1
                                 Системный администратор                    3
Кабельщики                       Кабельщик-сварщик                          2
                                 Монтажник СКС                              1
                                 Специалист по прокладке кабеля             3
                                 Техник по обслуживанию сетей               2
Связисты                         Инженер связи                       

In [8]:
events.head(20)

,id_event,title,description,created_at,deadline,completed_at,closed_at,status,priority,assigned_to,reported_by,id_department
0,1,Не работает внутренний номер,"Переименован в АТС, восстановлен доступ",2025-07-07 06:06:59.033255,2025-07-08 06:06:59.033255,2025-07-18 13:06:59.033255,2025-07-19 12:06:59.033255,отклонена,высокий,37,Хохлов Лавр Захарьевич,1
1,2,Не работает внутренний номер,"Переименован в АТС, восстановлен доступ",2025-06-26 00:46:39.210304,2025-06-29 00:46:39.210304,2025-07-01 17:46:39.210304,2025-07-03 12:46:39.210304,новая,низкий,46,Дьячков Дорофей Абрамович,1
2,3,Не работает внутренний номер,"Переименован в АТС, восстановлен доступ",2025-03-19 05:26:11.705513,2025-03-22 05:26:11.705513,2025-03-19 18:26:11.705513,2025-03-22 12:26:11.705513,выполнена,средний,31,Кошелева Фёкла Аскольдовна,1
3,4,Не работает телефон,"Заменен кабель, восстановлена связь",2025-07-08 07:38:35.692559,2025-07-22 07:38:35.692559,2025-07-16 07:38:35.692559,2025-07-16 14:38:35.692559,выполнена,низкий,34,Воробьев Харитон Васильевич,1
4,5,Не проходит вызов,"Проверен шлюз, ошибка устранена",2025-03-22 14:10:07.661486,2025-04-03 14:10:07.661486,2025-03-22 20:10:07.661486,2025-03-25 04:10:07.661486,выполнена,низкий,34,Аполлинарий Теймуразович Борисов,1
5,6,Нет сигнала в розетке,Перепатчена розетка на коммутаторе,2025-05-28 02:04:30.770978,2025-06-10 02:04:30.770978,2025-06-07 20:04:30.770978,2025-06-08 08:04:30.770978,в работе,низкий,51,Евфросиния Олеговна Кондратьева,1
6,7,Шумы в телефоне,Заменена гарнитура,2025-03-23 17:55:21.647690,2025-03-27 17:55:21.647690,2025-04-03 03:55:21.647690,2025-04-05 22:55:21.647690,в работе,низкий,47,Милий Федотович Терентьев,1
7,8,Не проходит вызов,"Проверен шлюз, ошибка устранена",2025-03-29 08:55:04.656909,2025-04-11 08:55:04.656909,2025-04-03 08:55:04.656909,2025-04-03 17:55:04.656909,выполнена,высокий,47,Фомина Олимпиада Ильинична,1
8,9,Не проходит вызов,"Проверен шлюз, ошибка устранена",2025-04-08 13:59:11.055938,2025-04-14 13:59:11.055938,2025-04-09 22:59:11.055938,2025-04-12 13:59:11.055938,выполнена,средний,10,Кабанова Прасковья Харитоновна,1
9,10,Шумы в телефоне,Заменена гарнитура,2025-04-01 14:41:14.885602,2025-04-04 14:41:14.885602,2025-04-06 15:41:14.885602,2025-04-06 18:41:14.885602,в работе,высокий,26,Мокей Бориславович Рыбаков,1


In [7]:
# Инициализация Faker с русской локализацией
fake = Faker('ru_RU')

# Создаём список из 10 уникальных сотрудников
team = [fake.name() for _ in range(10)]

# Определение списков для статичных данных
statuses = ['Ожидает подтверждения', 'Выполнена', 'Закрыт']
priorities = ['Низкий', 'Средний', 'Высокий']
locations = ['Офис 1', 'Офис 2', 'Офис 3', 'Офис 4', 'Офис 5', 'Офис 6', 'Офис 7', 'Офис 8', 'Офис 9', 'Офис 10']
services = ['Оргтехника', 'Информационная безопасность', 'Установка приложений']
equipments = ['Компьютер', 'Принтер', 'Монитор', 'Мышка', 'Клавиатура']
platforms = ['Площадка А', 'Площадка Б', 'Площадка В', 'Площадка Г']

# Словарь для планового времени выполнения в зависимости от приоритета
priority_days = {
    'Низкий': 15,
    'Средний': 10,
    'Высокий': 4
}

# Генерация данных
data = []
for i in range(1000):
    # Генерация даты регистрации
    reg_date = fake.date_time_between(start_date='-1y', end_date='now')
    
    # Выбор приоритета
    priority = random.choice(priorities)
    
    # Плановое время выполнения в зависимости от приоритета
    plan_days = priority_days[priority]
    plan_date = reg_date + timedelta(days=plan_days)
    
    # Статус
    status = random.choice(statuses)
    
    # Логика для фактического времени выполнения
    if status in ['Выполнена', 'Закрыт']:
        # 60% шанс, что заявка выполнена до планового времени
        if random.random() < 0.6:
            delta_days = random.uniform(0, plan_days)  # Между регистрацией и плановым временем
            actual_date = reg_date + timedelta(days=delta_days)
        else:
            # 40% шанс просрочки (до 10 дней после планового)
            delta_days = random.uniform(plan_days, plan_days + 10)
            actual_date = reg_date + timedelta(days=delta_days)
        actual_hours = (actual_date - reg_date).total_seconds() / 3600  # Время в часах
        overdue_status = 'Просрочен' if actual_date > plan_date else 'Непросрочен'
        resolved_by = random.choice(team) if random.random() < 0.95 else None
    else:  # Ожидает подтверждения
        actual_date = None
        actual_hours = None
        overdue_status = 'Непросрочен'
        resolved_by = None
    
    # Генерация номера заявки
    ticket_number = f'ITCC{random.randint(1000000, 9999999)}'
    
    # Генерация описания и решения
    description = random.choice([
        'Не печатает принтер', 'Ошибка доступа к сети', 'Не запускается приложение', 
        'Требуется установка ПО', 'Сломалась мышка', 'Проблема с монитором'
    ])
    resolution = random.choice([
        'Принтер перезагружен', 'Доступ восстановлен', 'ПО установлено', 
        'Мышка заменена', 'Монитор отремонтирован', 'Переустановлено приложение'
    ]) if status in ['Выполнена', 'Закрыт'] else 'В процессе'
    
    # Генерация конфигурационной единицы
    equipment_id = f'EQ{random.randint(1000, 9999)}'
    equipment_name = random.choice(equipments)
    
    # Создание записи
    row = {
        'Дата_время_регистрации': reg_date,
        'Плановое_время_выполнения': plan_date,
        'Фактическое_время_выполнения': actual_date,
        'Номер_заявки': ticket_number,
        'Статус': status,
        'Приоритет': priority,
        'Кем_решен': resolved_by,
        'Инициатор': fake.name(),
        'Расположение': random.choice(locations),
        'Описание': description,
        'Фактическое_время_выполнения_ч': round(actual_hours, 2) if actual_hours else None,
        'Услуга': random.choice(services),
        'Площадка_обслуживания': random.choice(platforms),
        'Описание_решения': resolution,
        'Конфигурационные_единицы': f'{equipment_id}: {equipment_name}',
        'Статус_просрочки': overdue_status
    }
    data.append(row)

# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение в CSV
df.to_csv('service_requests.csv', index=False, encoding='utf-8')

# Вывод первых строк для проверки
print(df.head())

# Проверка количества уникальных сотрудников и распределения статусов
print("\nУникальные сотрудники в поле 'Кем_решен':")
print(df[df['Кем_решен'].notnull()]['Кем_решен'].unique())
print("\nРаспределение статусов:")
print(df['Статус'].value_counts())
print("\nРаспределение статуса просрочки для 'Выполнена' и 'Закрыт':")
print(df[df['Статус'].isin(['Выполнена', 'Закрыт'])]['Статус_просрочки'].value_counts())
print("\nРаспределение планового времени по приоритетам:")
print(df.groupby('Приоритет')['Плановое_время_выполнения'].agg(lambda x: (x - df.loc[x.index, 'Дата_время_регистрации']).dt.days.describe()))

  Дата_время_регистрации Плановое_время_выполнения  \
0    2025-04-28 03:45:52       2025-05-08 03:45:52   
1    2024-07-30 08:18:27       2024-08-14 08:18:27   
2    2024-11-29 10:35:59       2024-12-14 10:35:59   
3    2025-03-03 18:41:25       2025-03-18 18:41:25   
4    2024-10-25 17:42:59       2024-11-09 17:42:59   

  Фактическое_время_выполнения Номер_заявки                 Статус Приоритет  \
0                          NaT  ITCC7472564  Ожидает подтверждения   Средний   
1   2024-08-12 20:14:54.079201  ITCC8557044                 Закрыт    Низкий   
2                          NaT  ITCC5961639  Ожидает подтверждения    Низкий   
3   2025-03-05 05:56:06.234775  ITCC9887737                 Закрыт    Низкий   
4   2024-11-16 04:41:06.923530  ITCC3777251                 Закрыт    Низкий   

                      Кем_решен                       Инициатор Расположение  \
0                          None         Михеев Антонин Изотович       Офис 9   
1     Силина Эмилия Рудольфовна  С

ValueError: Must produce aggregated value